## Семинар 5: Часть 2: Type Annotation, MynkeyType,  Virtual Environments, Packaging 

## Аннотация типов (Type Annotation)

Аннотации типов просто считываются интерпретатором Python и никак более не обрабатываются, если мы импортируем сторонний аннотированный код, то на нас это никак не повлияет. В первую очередь аннотация типов рассчитана на работу со статическими аналазаторами. 


В самом простом случае аннотацией содержит ожидамеый тип:

```python
name: str = Petya
```

Можем ли мы получить какую-то ошибку с аннотацией типов после фазы статистического анализа, при выполнении кода?





In [1]:
price: int 
price = '100$' # Incompatible types in assignment (только при стат. анализе!)

print(price)

100$


Мы можем аннотировать и параметры функций:

```python
def render_text_block(self, line: str) -> str:
    ...
```

### Optional
Если вы пемечаете переменную типов ```int``` и пытаетесь присвоить ей ```None```, будет ошибка ```Incompatible types```. Для таких случаем в модуле typing предусмотрен тип ```Optional``` (именно для возможности ```None```). 


In [25]:
from typing import Optional

def check_price(price: Optional[int] = None) -> bool:
    if price is None:
        return False
    else: 
        return price > 0

print(check_price())
print(check_price(-1))

False
False


### Any
Иногда вы не хотите ограничивать возможные типы переменной. Например, если это действительно не важно, или если вы планируете сделать обработку разных типов самостоятельно. В этом случае, можно использовать аннотацию ```Any```.

In [26]:
from typing import Any

smth: Any = 42 
smth = '42'

### Union

Для случаев, когда необходимо допустить использование не любых типов, а только некоторых, можно использовать аннотацию ```typing.Union``` с указанием списка типов в квадратных скобках.

In [28]:
from typing import Union

def square(x: Union[int, float]) -> Union[int, float]:
    return x ** 2

print(square(2))
print(square(2.0))
print(square("2")) # Argument 1 to "2" has incompatible type "str"; expected "Union[int, float]"

4
4.0


TypeError: unsupported operand type(s) for ** or pow(): 'str' and 'int'

### Списки (Lists)

Для того, чтобы указать, что переменная содержит список, мы можем просто указать ```list```, как ее тип.
```python
a: list = [1, 2, 3]
```
Однако, если нам нужно конкретизировать какие элементы этот список может содержать, нам нужно воспользоваться ```typing.List```:

In [15]:
from typing import List

authors: List[str] = ['James Joyce', 'Stephen King']

Пробуем создать матрешку из нескольких типов.
Пусть у нас есть ```example.py`` следующего содержания:
```python
from typing import List, Union

a: List[Union[int, float]] = [1.0, 2]
```

Теперь вызываем ```mypy```:
```bash
$ mypy example1.py

Success: no issues found in 1 source file
```

### Кортежи (Tuples)

Новым тут является то, что мы можем указать тип для каждой позиции кортежа:

In [18]:
from typing import Tuple

author_with_age: Tuple[str, int] = ('Stephen King', 73)

### Словари (Dicts)

In [19]:
from typing import Dict

book_authors: Dict[str, str] = {"Fahrenheit 451": "Bradbury"}

### Свои классы (Classes)

In [21]:
class MyClass:
    pass

foo: MyClass = MyClass()

### Статические анализаторы

Входят по-умолчанию во многие IDE и выдают подсказки в процессе набора. (см. [PyCharm](https://www.jetbrains.com/pycharm/)). 

Существую консольные линтеры (см. [PyLint](https://www.pylint.org/), см. [MyPy](https://github.com/python/mypy)). 

Исходник ```example.py```: 
```python
price: int 
price = '100$'
```

Проверка:
``` bash
$ mypy example.py

example.py:2: error: Incompatible types in assignment (expression has type "str", variable has type "int")
Found 1 error in 1 file (checked 1 source file)
```

Как поступить если у нас много кода без аннотации, но мы ценим время наших программистов?

### MonkeyType

[MonkeyType](https://github.com/Instagram/MonkeyType)  collects runtime types of function arguments and return values, and can automatically generate stub files or even add draft type annotations directly to your Python code based on the types collected at runtime.

## Виртуальные окружения (virtual environments)

Виртуальные окружения – это прежде всего изолированные среды, которые позволяют вам разрабатывать несколько проектов так, чтобы они не зависели друг от друга. 


Для пользователей Windows рекомендуется установить виртуально Ubuntu 20.04 LTS, сделать это можно через магазин приложений microsoft store. Установить python3 вам нужно самостоятельно 😉

Если у вас еще не установлен модуль venv: 
```bash
pip install virtualenv
```

#### Создание и использование виртуального окружения:
Создаем вирутальное оркужение:
```bash
python -m venv env
```

```env``` – папка в которой виртуальное окружение будет создано

Для активации вызываем команду:
```bash
source env/bin/activate
```

Когда окружение активировано команды установки пакетов будут действовать только для него. Давайте установим пакет monkeytype:
```bash
pip install monkeytype
```

Теперь посмотрим на список пакетов, которые появились в нашем окружении:

```bash
pip freeze
```

```bash
libcst==0.3.21
MonkeyType==21.5.0
mypy-extensions==0.4.3
PyYAML==5.4.1
typing-extensions==3.10.0.2
typing-inspect==0.7.1
```

#### Указание зависимостей для вашего приложения 

Если вы пишите какое-то приложение на python, то хорошим тоном считается добавление файла ```requirements.txt```, в котором указаны все необходимые зависиомсти.

Создать такой файл можно используя выход команды ```pip freeze```. 

Удобно установить все зависимости для приложения можно используя флаг ```-r```:

```bash
pip install -r requirements.txt
```

Сам файл при этом может выглядеть например вот так:
```

```

## Создание своих python пакетов (packaging)

Посмотрим на файл ```setup.py``` из репозитория MonkeyType:

```python
# Copyright (c) 2017-present, Facebook, Inc.
# All rights reserved.
#
# This source code is licensed under the BSD-style license found in the
# LICENSE file in the root directory of this source tree.

import os
from setuptools import setup, find_packages


def long_desc(root_path):
    FILES = ['README.rst', 'CHANGES.rst']
    for filename in FILES:
        filepath = os.path.realpath(os.path.join(root_path, filename))
        if os.path.isfile(filepath):
            with open(filepath, mode='r') as f:
                yield f.read()


HERE = os.path.abspath(os.path.dirname(__file__))
long_description = "\n\n".join(long_desc(HERE))


def get_version(root_path):
    with open(os.path.join(root_path, 'monkeytype', '__init__.py')) as f:
        for line in f:
            if line.startswith('__version__ ='):
                return line.split('=')[1].strip().strip('"\'')


setup(
    name='MonkeyType',
    version=get_version(HERE),
    license="BSD",
    description='Generating type annotations from sampled production types',
    long_description=long_description,
    author='Matt Page',
    author_email='mpage@instagram.com',
    url='https://github.com/instagram/MonkeyType',
    packages=find_packages(exclude=['tests*']),
    package_data={"monkeytype": ["py.typed"]},
    entry_points={
        'console_scripts': [
            'monkeytype=monkeytype.cli:entry_point_main'
        ]
    },
    python_requires='>=3.6',
    install_requires=['mypy_extensions', 'libcst>=0.3.7'],
    classifiers=[
        'Development Status :: 5 - Production/Stable',
        'Intended Audience :: Developers',
        'License :: OSI Approved :: BSD License',
        'Operating System :: OS Independent',
        'Programming Language :: Python',
        'Programming Language :: Python :: 3',
        'Programming Language :: Python :: 3.6',
        'Programming Language :: Python :: 3.7',
        'Programming Language :: Python :: 3.8',
        'Programming Language :: Python :: 3.9',
    ],
    zip_safe=False,
)
```